In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()
init.endpoint

replicating local resources


'https://m5knaekxo6.execute-api.us-west-2.amazonaws.com/dev-v0001/rlxmooc'

In [2]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"eduardobedoyamarin","key":"c5e22e55c7828572ad66be14edff8dd3"}'}

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d gregorut/videogamesales

  0% 0.00/381k [00:00<?, ?B/s]
100% 381k/381k [00:00<00:00, 75.6MB/s]


In [5]:
!unzip /content/videogamesales.zip

Archive:  /content/videogamesales.zip
  inflating: vgsales.csv             


In [95]:
#!wc vgsales.csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
df = pd.read_csv('vgsales.csv')
print("Tamaño: ", df.shape)
df.head(5)

Tamaño:  (16598, 11)


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [106]:
df.index = df.Name
del(df["Name"])
df.head()

,Rank,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Name,,,,,,,,,,
Wii Sports,1,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
Super Mario Bros.,2,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
Mario Kart Wii,3,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
Wii Sports Resort,4,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
Pokemon Red/Pokemon Blue,5,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [108]:
for c in df.columns:
    print ("%15a"%c, df[c].dtype)

         'Rank' int64
     'Platform' object
         'Year' float64
        'Genre' object
    'Publisher' object
     'NA_Sales' float64
     'EU_Sales' float64
     'JP_Sales' float64
  'Other_Sales' float64
 'Global_Sales' float64


In [111]:
df._get_numeric_data().describe().T

,count,mean,std,min,25%,50%,75%,max
Rank,16598.0,8300.605254,4791.853933,1.00,4151.25,8300.50,12449.75,16600.00
Year,16327.0,2006.406443,5.828981,1980.00,2003.00,2007.00,2010.00,2020.00
NA_Sales,16598.0,0.264667,0.816683,0.00,0.00,0.08,0.24,41.49
EU_Sales,16598.0,0.146652,0.505351,0.00,0.00,0.02,0.11,29.02
JP_Sales,16598.0,0.077782,0.309291,0.00,0.00,0.00,0.04,10.22
Other_Sales,16598.0,0.048063,0.188588,0.00,0.00,0.01,0.04,10.57
Global_Sales,16598.0,0.537441,1.555028,0.01,0.06,0.17,0.47,82.74


In [102]:
#df._get_numeric_data().columns
df.columns

Index(['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'],
      dtype='object')

In [112]:
#Valores faltantes
missing = df.isna().sum()
#print(missing, '\n')
print(missing[missing!=0])
#df.columns

Year         271
Publisher     58
dtype: int64


In [105]:
cols1 = ['Rank', 'Year', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
cols3 = ['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Global_Sales']
sns.set()
#sns.pairplot(df[cols1])

In [121]:
#df.Year.fillna(df.Year.mean(), inplace=True)
dn = df.T.dropna().T
dn = dn[[i for i in dn.columns if df[i].dtype!=object]]
#print (dn.shape)
dlr = dn.copy()
aux = df["Year"].copy()
aux[aux.isna()] = np.random.normal(loc=np.mean(aux), scale=np.std(aux), size=np.sum(aux.isna()))
dlr["Year"] = aux
dlr

,Rank,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Year
Name,,,,,,,
Wii Sports,1,41.49,29.02,3.77,8.46,82.74,2006.0
Super Mario Bros.,2,29.08,3.58,6.81,0.77,40.24,1985.0
Mario Kart Wii,3,15.85,12.88,3.79,3.31,35.82,2008.0
Wii Sports Resort,4,15.75,11.01,3.28,2.96,33.0,2009.0
Pokemon Red/Pokemon Blue,5,11.27,8.89,10.22,1.0,31.37,1996.0
...,...,...,...,...,...,...,...
Woody Woodpecker in Crazy Castle 5,16596,0.01,0.0,0.0,0.0,0.01,2002.0
Men in Black II: Alien Escape,16597,0.01,0.0,0.0,0.0,0.01,2003.0
SCORE International Baja 1000: The Official Game,16598,0.0,0.0,0.0,0.0,0.01,2008.0


In [120]:
#Valores faltantes
missingg = dlr.isna().sum()
#print(missing, '\n')
missingg

Rank            0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
Year            0
dtype: int64

In [143]:
dlr['Year'] = dlr['Year'].astype(np.int64)
print(dlr.dtypes)
print(dlr['Year'].unique())
len(dlr['Year'].unique())

Rank            object
NA_Sales        object
EU_Sales        object
JP_Sales        object
Other_Sales     object
Global_Sales    object
Year             int64
dtype: object
[2006 1985 2008 2009 1996 1989 1984 2005 1999 2007 2010 2013 2004 1990
 1988 2002 2001 2011 1998 2015 2012 2014 1992 1997 1993 1994 1982 2003
 1986 2000 1995 2016 1991 1981 1987 1980 1983 2018 2019 2020 2024 2017]


42